### Project 6 - Solution

```
1. Supplier exceptions
    a. Not manufactured anymore
    b. Production delayed
    c. Shipping delayed
    
2. Checkout exceptions
    a. Inventory type exceptions
        - out of stock
    b. Pricing exceptions
        - invalid coupon code
        - cannot stack coupons
```

In [1]:
from datetime import datetime 

class WidgetException(Exception):
    message = 'Generic Widget exception.'
    
    def __init__(self, *args, customer_message=None):
        super().__init__(args)
        if args:
            self.message = args[0]
        self.customer_message = customer_message if customer_message is not None else self.message
        
    def log_exception(self):
        exception = {
            "type": type(self).__name__,
            "message": self.message,
            "args": self.args[1:]
        }
        print(f'EXCEPTION: {datetime.utcnow().isoformat()}: {exception}')

In [2]:
ex1 = WidgetException('some custom message', 10, 100)
ex2 = WidgetException(customer_message='A custom user message.')

In [3]:
ex1.log_exception()

EXCEPTION: 2019-08-15T05:25:05.724235: {'type': 'WidgetException', 'message': 'some custom message', 'args': ()}


In [4]:
ex2.log_exception()

EXCEPTION: 2019-08-15T05:25:05.732242: {'type': 'WidgetException', 'message': 'Generic Widget exception.', 'args': ()}


Now we can create our hierarchy, and override the appropriate values for `message` to make it more specific:

In [5]:
class SupplierException(WidgetException):
    message = 'Supplier exception.'

class NotManufacturedException(SupplierException):
    message = 'Widget is no longer manufactured by supplier.'
    
class ProductionDelayedException(SupplierException):
    message = 'Widget production has been delayed by supplier.'
    
class ShippingDelayedException(SupplierException):
    message = 'Widget shipping has been delayed by supplier.'
    
class CheckoutException(WidgetException):
    message = 'Checkout exception.'
    
class InventoryException(CheckoutException):
    message = 'Checkout inventory exception.'
    
class OutOfStockException(InventoryException):
    message = 'Inventory out of stock'
    
class PricingException(CheckoutException):
    message = 'Checkout pricing exception.'
    
class InvalidCouponCodeException(PricingException):
    message = 'Invalid checkout coupon code.'
    
class CannotStackCouponException(PricingException):
    message = 'Cannot stack checkout coupon codes.'

And now we can use any of these exceptions in our code, and use the defined "logger" we implemented:

In [6]:
try:
    raise CannotStackCouponException()
except WidgetException as ex:
    ex.log_exception()
    raise

EXCEPTION: 2019-08-15T05:25:05.748971: {'type': 'CannotStackCouponException', 'message': 'Cannot stack checkout coupon codes.', 'args': ()}


CannotStackCouponException: ()

Next let's add the http status codes we want to assign to each exception type.

In [7]:
from http import HTTPStatus

In [8]:
class WidgetException(Exception):
    message = 'Generic Widget exception.'
    http_status = HTTPStatus.INTERNAL_SERVER_ERROR
    
    def __init__(self, *args, customer_message=None):
        super().__init__(*args)
        if args:
            self.message = args[0]
        self.customer_message = customer_message if customer_message is not None else self.message
        
    def log_exception(self):
        exception = {
            "type": type(self).__name__,
            "message": self.message,
            "args": self.args[1:]
        }
        print(f'EXCEPTION: {datetime.utcnow().isoformat()}: {exception}')

Before we redefine our child classes, let's also implement the `to_json` function that we can use to send back to our users:

In [9]:
import json

In [10]:
class WidgetException(Exception):
    message = 'Generic Widget exception.'
    http_status = HTTPStatus.INTERNAL_SERVER_ERROR
    
    def __init__(self, *args, customer_message=None):
        super().__init__(*args)
        if args:
            self.message = args[0]
        self.customer_message = customer_message if customer_message is not None else self.message
        
    def log_exception(self):
        exception = {
            "type": type(self).__name__,
            "message": self.message,
            "args": self.args[1:]
        }
        print(f'EXCEPTION: {datetime.utcnow().isoformat()}: {exception}')
        
    def to_json(self):
        response = {
            'code': self.http_status.value,
            'message': '{}: {}'.format(self.http_status.phrase, self.customer_message),
            'category': type(self).__name__,
            'time_utc': datetime.utcnow().isoformat()            
        }
        return json.dumps(response)

In [11]:
e = WidgetException('same custom message for log and user')

In [12]:
e.log_exception()

EXCEPTION: 2019-08-15T05:25:13.484482: {'type': 'WidgetException', 'message': 'same custom message for log and user', 'args': ()}


In [13]:
json.loads(e.to_json())

{'code': 500,
 'message': 'Internal Server Error: same custom message for log and user',
 'category': 'WidgetException',
 'time_utc': '2019-08-15T05:25:13.650056'}

In [14]:
e = WidgetException('custom internal message', customer_message='custom user message')

In [15]:
e.log_exception()

EXCEPTION: 2019-08-15T05:25:13.973345: {'type': 'WidgetException', 'message': 'custom internal message', 'args': ()}


In [16]:
e.to_json()

'{"code": 500, "message": "Internal Server Error: custom user message", "category": "WidgetException", "time_utc": "2019-08-15T05:25:14.136676"}'

Now for the bonus exercise - I asked you to try and log the stack trace as well.

To do that we could cannot simply use the `str` or `repr` of the  `__traceback__` property of the exception:

In [17]:
try:
    raise WidgetException('custom error message')
except WidgetException as ex:
    print(repr(ex.__traceback__))

Instead we can use the `traceback` module:

In [18]:
import traceback

In [19]:
try:
    raise ValueError
except ValueError:
    try:
        raise WidgetException('custom error message')
    except WidgetException as ex:
        print(list(traceback.TracebackException.from_exception(ex).format()))

['Traceback (most recent call last):\n', '  File "<ipython-input-19-2a9225338511>", line 2, in <module>\n    raise ValueError\n', 'ValueError\n', '\nDuring handling of the above exception, another exception occurred:\n\n', 'Traceback (most recent call last):\n', '  File "<ipython-input-19-2a9225338511>", line 5, in <module>\n    raise WidgetException(\'custom error message\')\n', 'WidgetException: custom error message\n']


So we can use that to implement logging the traceback. What would be nice too would be to expose the formatted traceback in our exception class while we're at it.

Since tracebacks can be huge, we're not going to materialize the traceback generator in that property (we'll still have to when we log the exception):

In [20]:
class WidgetException(Exception):
    message = 'Generic Widget exception.'
    http_status = HTTPStatus.INTERNAL_SERVER_ERROR
    
    def __init__(self, *args, customer_message=None):
        super().__init__(*args)
        if args:
            self.message = args[0]
        self.customer_message = customer_message if customer_message is not None else self.message
        
    @property
    def traceback(self):
        return traceback.TracebackException.from_exception(self).format()
    
    def log_exception(self):
        exception = {
            "type": type(self).__name__,
            "message": self.message,
            "args": self.args[1:],
            "traceback": list(self.traceback)
        }
        print(f'EXCEPTION: {datetime.utcnow().isoformat()}: {exception}')
        
    def to_json(self):
        response = {
            'code': self.http_status.value,
            'message': '{}: {}'.format(self.http_status.phrase, self.customer_message),
            'category': type(self).__name__,
            'time_utc': datetime.utcnow().isoformat()            
        }
        return json.dumps(response)

In [21]:
try:
    raise WidgetException('custom internal message', customer_message='custom user message')
except WidgetException as ex:
    ex.log_exception()
    print('------------')
    print(ex.to_json())

EXCEPTION: 2019-08-15T05:25:15.569467: {'type': 'WidgetException', 'message': 'custom internal message', 'args': (), 'traceback': ['Traceback (most recent call last):\n', '  File "<ipython-input-21-472686457160>", line 2, in <module>\n    raise WidgetException(\'custom internal message\', customer_message=\'custom user message\')\n', 'WidgetException: custom internal message\n']}
------------
{"code": 500, "message": "Internal Server Error: custom user message", "category": "WidgetException", "time_utc": "2019-08-15T05:25:15.569634"}


What's nice now, is that we could just print the traceback wihout logging the exception:

In [22]:
try:
    a = 1 / 0
except ZeroDivisionError:
    try:
        raise WidgetException()
    except WidgetException as ex:
        print(''.join(ex.traceback))

Traceback (most recent call last):
  File "<ipython-input-22-2212fef7bb30>", line 2, in <module>
    a = 1 / 0
ZeroDivisionError: division by zero

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "<ipython-input-22-2212fef7bb30>", line 5, in <module>
    raise WidgetException()
WidgetException



Now we can define our exception sub types, including the http status for each:

In [23]:
class SupplierException(WidgetException):
    message = 'Supplier exception.'
    http_status = HTTPStatus.INTERNAL_SERVER_ERROR

class NotManufacturedException(SupplierException):
    message = 'Widget is no longer manufactured by supplier.'
    http_status = HTTPStatus.INTERNAL_SERVER_ERROR
    
class ProductionDelayedException(SupplierException):
    message = 'Widget production has been delayed by supplier.'
    http_status = HTTPStatus.INTERNAL_SERVER_ERROR
    
class ShippingDelayedException(SupplierException):
    message = 'Widget shipping has been delayed by supplier.'
    http_status = HTTPStatus.INTERNAL_SERVER_ERROR
    
class CheckoutException(WidgetException):
    message = 'Checkout exception.'
    http_status = HTTPStatus.INTERNAL_SERVER_ERROR
    
class InventoryException(CheckoutException):
    message = 'Checkout inventory exception.'
    http_status = HTTPStatus.INTERNAL_SERVER_ERROR
    
class OutOfStockException(InventoryException):
    message = 'Inventory out of stock'
    http_status = HTTPStatus.INTERNAL_SERVER_ERROR
    
class PricingException(CheckoutException):
    message = 'Checkout pricing exception.'
    http_status = HTTPStatus.INTERNAL_SERVER_ERROR
    
class InvalidCouponCodeException(PricingException):
    message = 'Invalid checkout coupon code.'
    http_status = HTTPStatus.BAD_REQUEST
    
class CannotStackCouponException(PricingException):
    message = 'Cannot stack checkout coupon codes.'
    http_status = HTTPStatus.BAD_REQUEST

In [24]:
e = InvalidCouponCodeException('User tried to use an old coupon', customer_message='Sorry. This coupon has expired.')

In [25]:
e.log_exception()

EXCEPTION: 2019-08-15T05:25:16.939141: {'type': 'InvalidCouponCodeException', 'message': 'User tried to use an old coupon', 'args': (), 'traceback': ['InvalidCouponCodeException: User tried to use an old coupon\n']}


In [26]:
e.to_json()

'{"code": 400, "message": "Bad Request: Sorry. This coupon has expired.", "category": "InvalidCouponCodeException", "time_utc": "2019-08-15T05:25:17.108099"}'

As you can see our traceback was empty above (the exception is present, but there is no call stack) - because we did not actually raise the exception!

In [27]:
try:
    raise ValueError
except ValueError:
    try:
        raise InvalidCouponCodeException(
            'User tried to use an old coupon', customer_message='Sorry. This coupon has expired.'
        )
    except InvalidCouponCodeException as ex:
        ex.log_exception()
        print('------------')
        print(ex.to_json())
        print('------------')
        print(''.join(ex.traceback))

EXCEPTION: 2019-08-15T05:25:17.441852: {'type': 'InvalidCouponCodeException', 'message': 'User tried to use an old coupon', 'args': (), 'traceback': ['Traceback (most recent call last):\n', '  File "<ipython-input-27-775351168ae0>", line 2, in <module>\n    raise ValueError\n', 'ValueError\n', '\nDuring handling of the above exception, another exception occurred:\n\n', 'Traceback (most recent call last):\n', '  File "<ipython-input-27-775351168ae0>", line 6, in <module>\n    \'User tried to use an old coupon\', customer_message=\'Sorry. This coupon has expired.\'\n', 'InvalidCouponCodeException: User tried to use an old coupon\n']}
------------
{"code": 400, "message": "Bad Request: Sorry. This coupon has expired.", "category": "InvalidCouponCodeException", "time_utc": "2019-08-15T05:25:17.442103"}
------------
Traceback (most recent call last):
  File "<ipython-input-27-775351168ae0>", line 2, in <module>
    raise ValueError
ValueError

During handling of the above exception, another